# Classification de texte avec NLTK et Text Blob

## Analyse sentimentale avec NLTK

Dans un premier temps, on télécharge un corpus contenant des commentaires sur des films et qui sont annotés; c'est à dire pour chaque commentaire on a l'info (sous forme de flag  ou 1) si c'est positif ou négatif...

In [1]:
import nltk
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/bestprojectever/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [2]:
from nltk.corpus import movie_reviews

On effectue une petite transformation (Reshaping) de données pour les préparer en entrée de notre algorithme de classification...

In [3]:
documents = [(list(movie_reviews.words(fileid)), category)
              for category in movie_reviews.categories() for fileid in movie_reviews.fileids(category)]

In [5]:
len(documents)

2000

Notre dataset contient 2000 commentaires

Un petit <b> shuffle </b>  est nécessaire avant le Train Test split pour avoir un entraînement et un test qui ne dépendent pas de la séquence de ces commentaires...

In [7]:
import random
random.shuffle(documents)

In [8]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())

In [9]:
all_words

FreqDist({',': 77717, 'the': 76529, '.': 65876, 'a': 38106, 'and': 35576, 'of': 34123, 'to': 31937, "'": 30585, 'is': 25195, 'in': 21822, ...})

On se restreint comme nombre de caractéristiques ou Features cible de la numérisation par exemple à 2000 (les 2000 mots les plus fréquents...). Vous pouvez changer ce nombre...

In [10]:
word_features = list(all_words.keys())[:2000]

Pour la numérisation, on fait appel à un procédé qui à chaque commentaire associe un vecteur de caractéristiques dont les composantes correspondent aux mots les plus fréquents calculés précédemment <br>
La valeur de la composante est égale à 1 si le mot est présent dans le commentaire, 0 sinon

In [11]:
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [12]:
#print(document_features(movie_reviews.words('pos/cv957_8737.txt')))

In [13]:
featuresets = [(document_features(d), c) for (d,c) in documents]

On effectue le Train Test Split en attribuant les 100 première commentaires à l'ensemble de test et le reste (1900 commentaires) à l'ensemble de Train (apprentissage)

In [14]:
train_set, test_set = featuresets[100:], featuresets[:100]

On importe le modèle de SVM linéaire depuis Scikit-Learn et qui sera encapsulé par NLTK

In [15]:
from sklearn.svm import LinearSVC

On entraîne le modèle SVM linéaire sur les données d'apprentissage

In [16]:
classifier = nltk.classify.SklearnClassifier(LinearSVC()).train(train_set)

/home/bestprojectever/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Voilà la classification faite pour le premier échantillon de l'ensemble de test

In [19]:
## Décommentez la ligne suivante pour afficher le contenu du commentaire et vous rendre compte de son caractère négatof
#' '.join(documents[0][0])
classifier.classify(test_set[0][0])

'neg'

On effectue des prédictions sur tout l'ensemble de test et on calcule un score de précision (accuracy) comme étant le rapport du nombre de bonnes prédictions sur la taille de l'ensemble <br>
Ce calcul s'effectue grâce à la fonction <b> nltk.classify.accuracy </b> 

In [21]:
print(nltk.classify.accuracy(classifier, test_set))

0.69


## Analyse sentimentale avec TextBlob

TextBlob fournit une fonctionnalité se basant sur un modèle déjà entraîné pour prédire si un texte est négatif ou positif

In [22]:
from textblob import TextBlob
testimonial = TextBlob("Textblob is amazingly simple to use. What great fun!")

L'attribut <b> sentiment </b> de l'objet <b> TextBlob </b> consiste en un tuple de la forme <b> (polarité, subjectivité) </b> où la polarité est un réel <b> float </b> compris entre -1.0 et 1.0 (-1.0 pour les textes les plus négatifs et 1.0 pour les textes les plus positifs) et la subjectivité est un réel <b> float </b> compris entre 0.0 et 1.0 où 0.0 veut dire <b> très objectif </b> et 1.0 veut dire <b> très subjectif </b>

In [23]:
testimonial.sentiment

Sentiment(polarity=0.39166666666666666, subjectivity=0.4357142857142857)